In [ ]:
%load_ext autoreload
%autoreload 2

from setup_imports import *  # noqa: F401,F403

from src.phrases.generation import generate_phrases_from_vocab_dict
from src.utils import (
    load_json,
    save_text_file,
    get_longman_verb_vocab_dict,
    load_text_file,
)
from src.phrases.phrase_model import Phrase

# Phrase Generation

We will have short, simple verb phrases, and associated vocab-only phrases from a vocab_dict. The deliberate removal of verbs from vocab phrases is designed to make the cards easier, and split those tasks

In [ ]:
vocab_dict = load_json("..\data\gcse_vocab_list_cambridge.json")

In [ ]:
first2000 = get_longman_verb_vocab_dict(
    "..\data\longman-communication-3000.json", category="S2"
)

In [ ]:
# let's create then shuffle-up verbs and vocab

# from random import shuffle


# all_verbs = (first2000['verbs'])
# all_vocab = (first2000['vocab'])
# shuffle(all_verbs)
# shuffle(all_vocab)

In [ ]:
# save_text_file(all_verbs, "../data/LM2000_verbs.txt")
# save_text_file(all_vocab, "../data/LM2000_vocab.txt")



In [ ]:
all_verbs = load_text_file("../data/LM2000_verbs.txt")
all_vocab = load_text_file("../data/LM2000_vocab.txt")
print(f"num verbs: {len(all_verbs)}, num vocab: {len(all_vocab)}")

In [ ]:
# aim for 10 verbs and 50 vocab
FROM_INDEX = 20
TO_INDEX = FROM_INDEX + 10
VOCAB_FROM_INDEX = int((FROM_INDEX / 10) * 50)
VOCAB_TO_INDEX = VOCAB_FROM_INDEX + 50


COLLECTION = "LM2000"
DECK = "Pack03"
some_verbs = all_verbs[FROM_INDEX:TO_INDEX]
some_vocab = all_vocab[VOCAB_FROM_INDEX:VOCAB_TO_INDEX]

current_dict = {}
current_dict["verbs"] = list(some_verbs)
current_dict["vocab"] = list(some_vocab)

print(
    f"FROM_INDEX: {FROM_INDEX}, TO_INDEX: {TO_INDEX}, VOCAB_FROM_INDEX: {VOCAB_FROM_INDEX}, VOCAB_TO_INDEX: {VOCAB_TO_INDEX}, len verbs: {len(some_verbs)}, len vocab: {len(some_vocab)}"
)

In [ ]:
generated_phrases = generate_phrases_from_vocab_dict(
    current_dict,
    max_iterations=20,
)

In [ ]:
generated_phrases[0]

In [ ]:
save_text_file(generated_phrases[0], f"..\data\LM2000-{FROM_INDEX}-{TO_INDEX}.txt")

In [ ]:
generated_phrases = load_text_file(f"..\data\LM2000-{FROM_INDEX}-{TO_INDEX}.txt")

In [ ]:
generated_phrases = load_text_file(Path("..\data\phrases\\biology_lesson_test.txt"))
COLLECTION = "Biology"
DECK = "Sample"

In [ ]:
ALL_PHRASES = []
for phrase in generated_phrases:
    p = Phrase.create(phrase)
    p.collection = COLLECTION
    p.deck = DECK
    p.generate_image()
    #p.translate("uk-UA", refine=True, overwrite=True)
    p.generate_audio(context="flashcard", language="en-GB")
    p.upload(overwrite=True)
    ALL_PHRASES.append(p)

In [ ]:
ALL_PHRASES[0].tokens